In [ ]:
#uses DotProduct_Systolic_#N - v1.1x
#uses DotProduct_#N_noHR_C#mamcs_A#baseAdderSize - v3.7
#uses DotProduct_#N_noHR_M_A#baseAdderSize - v3.7
#uses RegisteredMultiplier  - v1.0x

In [ ]:
import sys
import math

In [ ]:
#N>=2
N = input('Enter N: ')

In [ ]:
N = int(N)
if(N<2):
    N=2

In [ ]:
NOEPS = input('Enter No Of Elements Per Series (>=1, <={}, empty for {}): '.format(N-1, N-1))

In [ ]:
if not NOEPS:
    NMult=N-1
else:
    NMult = int(NOEPS)
    if NMult<1:
        NMult=1
    elif NMult>(N-1):
        NMult=N-1

In [ ]:
#MAMCS>=1
MAMCS = input('Enter Multiply-Add Max Component Size (empty for {}): '.format(NMult))

In [ ]:
if not MAMCS:
    MAMCS = NMult
else:
    MAMCS = int(MAMCS)
    if MAMCS>NMult:
        MAMCS = NMult;
    elif(MAMCS<1):
            MAMCS=1

In [ ]:
#AdderSize>=2
if MAMCS<NMult:
    AdderSize = input('Enter maximum size allowed for base adders (empty for 2): ')

In [ ]:
#IN_WIDTH>=1
IN_WIDTH = input('Enter IN_WIDTH Default Value (empty for 10): ')

In [ ]:
#INPUT_REG_DEPTH>=0
INPUT_REG_DEPTH = input('Enter INPUT_REG_DEPTH Default Value (empty for 1): ')

In [ ]:
#MULT_PIPE_DEPTH>=0
MULT_PIPE_DEPTH = input('Enter MULT_PIPE_DEPTH Default Value (empty for 1): ')

In [ ]:
HRR = math.ceil(N/NMult);

NumOfComponents = math.ceil(NMult/MAMCS)
if MAMCS==NMult:
    AdderSize=0
else:
    if not AdderSize:
        AdderSize=2
    else:
        AdderSize = int(AdderSize)
        if AdderSize<2:
            AdderSize=2
        if AdderSize>NumOfComponents:
            AdderSize=NumOfComponents

if not IN_WIDTH:
    IN_WIDTH=10
else:
    IN_WIDTH = int(IN_WIDTH)
    if(IN_WIDTH<1):
        IN_WIDTH=1

if not INPUT_REG_DEPTH:
    INPUT_REG_DEPTH=1
else:
    INPUT_REG_DEPTH = int(INPUT_REG_DEPTH)
    if(INPUT_REG_DEPTH<0):
        INPUT_REG_DEPTH=0

if not MULT_PIPE_DEPTH:
    MULT_PIPE_DEPTH=1
else:
    MULT_PIPE_DEPTH = int(MULT_PIPE_DEPTH)
    if(MULT_PIPE_DEPTH<0):
        MULT_PIPE_DEPTH=0

In [ ]:
_Use_Registered_earlyOutReady_If_Available = 1

# _Use_Registered_earlyOutReady_If_Available = input('Use Registered earlyOutReady If Available? (empty or 1 to enable, 0 to disable): ')
# if not _Use_Registered_earlyOutReady_If_Available:
#     _Use_Registered_earlyOutReady_If_Available = 1
# else:
#     _Use_Registered_earlyOutReady_If_Available = int(_Use_Registered_earlyOutReady_If_Available)

In [ ]:
ModuleName="DotProduct_{}_S{}E_HRx{}_C{}_A{}".format(N, NMult, HRR, MAMCS, AdderSize)

In [ ]:
__Print_To_File = 0

if __Print_To_File<=0:
    of=sys.stdout
else:
    of=open("./"+ModuleName+".v", 'w+')

In [ ]:
print("`timescale 1ns / 1ps\n", file=of)
print("module "+ModuleName, file=of)
print("#(", file=of)
print("parameter IN_WIDTH = {},".format(IN_WIDTH), file=of)
print("parameter INPUT_REG_DEPTH = {},".format(INPUT_REG_DEPTH), file=of)
print("parameter MULT_PIPE_DEPTH = {}".format(MULT_PIPE_DEPTH), file=of)
print(")(", file=of)
print("input clk, reset, enable,", file=of)
print("output reg newInSeriesStart = 1,", file=of)
lghrr=math.ceil(math.log2(HRR))
if lghrr==1:
    print("output reg inSeries = 0,", file=of)
else:
    print("output reg [{}:0] inSeries = 0,".format(lghrr-1), file=of)
print("input inReady,", file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(NMult):
    print("A{}, ".format(i), end='', file=of)
print(file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(NMult):
    print("B{}, ".format(i), end='', file=of)
print(file=of)
print("output reg outReady = 0,", file=of)
lgn=math.ceil(math.log2(N))
al=-1+lgn
if al==0:
    print("output reg signed [(2*IN_WIDTH):0] DP,", file=of)
else:
    print("output reg signed [(2*IN_WIDTH)+{}:0] DP,".format(al), file=of)
if _Use_Registered_earlyOutReady_If_Available<=0:
    print("output earlyOutReady", file=of)    
else:
    print("output reg earlyOutReady", file=of)
print(");\n", file=of)

In [ ]:
print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(NMult-1):
    print("SA{}, ".format(i), end='', file=of)
print("SA{};".format(NMult-1), file=of)
print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(NMult-1):
    print("SB{}, ".format(i), end='', file=of)
print("SB{};".format(NMult-1), file=of)
print(file=of)

In [ ]:
palA=-1+math.ceil(math.log2(NMult))
if palA==0:
    print("wire signed [(2*IN_WIDTH):0] DPpart;", file=of)
elif palA<0:
    print("wire signed [(2*IN_WIDTH){}:0] DPpart;".format(palA), file=of)
else:
    print("wire signed [(2*IN_WIDTH)+{}:0] DPpart;".format(palA), file=of)
print("wire DPOutReady, DPEarlyOutReady;", file=of)
if NMult==1:
    print("RegisteredMultiplier", file=of)
elif MAMCS==NMult:
    print("DotProduct_Systolic_{}".format(NMult), file=of)
elif MAMCS==1:
    print("DotProduct_{}_noHR_M_A{}".format(NMult, AdderSize), file=of)  
else:
    print("DotProduct_{}_noHR_C{}_A{}".format(NMult, MAMCS, AdderSize), file=of)
print("#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )", file=of)
print("DPE (clk, reset, enable,", file=of)
print("inReady,", file=of)
for j in range(NMult):
    print("SA{}, ".format(j), end='', file=of)
print(file=of)
for j in range(NMult):
    print("SB{}, ".format(j), end='', file=of)
print(file=of)
print("DPOutReady,", file=of)
print("DPpart,", file=of)
print("DPEarlyOutReady);", file=of)
print(file=of)

In [ ]:
print("always @(posedge clk) begin", file=of)
print("\tif(reset) begin", file=of)
print("\t\tnewInSeriesStart <= 1;", file=of)
print("\t\tinSeries <= 0;", file=of)
print("\tend", file=of)
print("\telse if(enable) begin", file=of)
print("\t\tif(inReady) begin", file=of)
print("\t\t\tif(inSeries=={}) begin".format(HRR-1), file=of)
print("\t\t\t\tnewInSeriesStart <= 1;", file=of)
print("\t\t\t\tinSeries <= 0;", file=of)
print("\t\t\tend", file=of)
print("\t\t\telse begin", file=of)
print("\t\t\t\tnewInSeriesStart <= 0;", file=of)
if HRR==2:
    print("\t\t\t\tinSeries <= 1;", file=of)    
else:
    print("\t\t\t\tinSeries <= inSeries+1;", file=of)
print("\t\t\tend", file=of)
print("\t\tend", file=of)
print("\tend", file=of)
print("end", file=of)
print(file=of)

In [ ]:
#NZ = HRR*NMult - N
nzs=int(N/NMult); #number of no zero input to internal DP series
print("always @(*) begin", file=of)
if HRR!=nzs:
    print("\tif(inSeries!={}) begin\n\t".format(HRR-1), end='', file=of)
print("\t", end='', file=of)
for j in range(NMult):
    print("SA{} = A{}; ".format(j,j), end='', file=of)
print(file=of)
if HRR!=nzs:
    print("\t", end='', file=of)
print("\t", end='', file=of)
for j in range(NMult):
    print("SB{} = B{}; ".format(j,j), end='', file=of)
print(file=of)
if HRR!=nzs:
    print("\tend", file=of)
    print("\telse begin", file=of)
    print("\t\t", end='', file=of)
    for j in range(NMult):
        k=NMult*nzs+j
        if k >= N:
             print("SA{} = 0; ".format(j), end='', file=of)            
        else:
            print("SA{} = A{}; ".format(j,j), end='', file=of)
    print("\n\t\t", end='', file=of)
    for j in range(NMult):
        k=NMult*nzs+j
        if k >= N:
            print("SB{} = 0; ".format(j), end='', file=of)            
        else:
            print("SB{} = B{}; ".format(j,j), end='', file=of)
    print("\n\tend", file=of)
print("end", file=of)
print(file=of)

In [ ]:
if lghrr==1:
    print("reg outSeries = 0;", file=of)
else:
    print("reg [{}:0] outSeries = 0;".format(lghrr-1), file=of)
if al==0:
    print("reg signed [(2*IN_WIDTH):0] DPps = 0;", file=of)
else:
    print("reg signed [(2*IN_WIDTH)+{}:0] DPps = 0;".format(al), file=of)
print("always @(posedge clk) begin", file=of)
print("\tif(reset) begin", file=of)
print("\t\toutSeries <= 0;", file=of)
print("\t\tDPps <= 0;", file=of)
print("\t\toutReady <= 0;", file=of)
print("\tend", file=of)

print("\telse if(enable) begin", file=of)
print("\t\tif(outSeries=={}) begin".format(HRR-1), file=of)
print("\t\t\tif(DPOutReady) begin", file=of)
print("\t\t\t\toutReady <= 1;", file=of)
print("\t\t\t\tDP <= DPps+DPpart;", file=of)
print("\t\t\t\toutSeries <= 0;", file=of)
print("\t\t\tend", file=of)
print("\t\tend", file=of)
print("\t\telse begin", file=of)
print("\t\t\toutReady <= 0;", file=of)
print("\t\t\tif(DPOutReady) begin", file=of)
if HRR==2:
    print("\t\t\t\tDPps <= DPpart;", file=of)
    print("\t\t\t\toutSeries <= 1;", file=of)
else:
    print("\t\t\t\tif(outSeries==0) begin", file=of)
    print("\t\t\t\t\tDPps <= DPpart;", file=of)
    print("\t\t\t\tend", file=of)
    print("\t\t\t\telse begin", file=of)
    print("\t\t\t\t\tDPps <= DPps+DPpart;", file=of)
    print("\t\t\t\tend", file=of)
    print("\t\t\t\toutSeries <= outSeries+1;", file=of)
print("\t\t\tend", file=of)
print("\t\tend", file=of)
print("\tend", file=of)
print("end\n", file=of)

In [ ]:
if _Use_Registered_earlyOutReady_If_Available<=0:
    print("assign earlyOutReady = (outSeries=={}) & DPOutReady;\n".format(HRR-1), file=of)
else:
    if NMult==1:
        #combinational earlyOutReady:
        print("generate", file=of)
        print("if(MULT_PIPE_DEPTH==0) begin\n", file=of)

        print("always @(*) earlyOutReady = (outSeries=={}) & DPOutReady;\n".format(HRR-1), file=of)
        print("end", file=of)
        print("else begin\n", file=of)

    #registered earlyOutReady:
    print("initial earlyOutReady = 0;", file=of)
    print("always @(posedge clk) begin", file=of)
    print("\tif(reset) begin", file=of)
    print("\t\tearlyOutReady <= 0;", file=of)
    print("\tend", file=of)
    print("\telse if(enable) begin", file=of)
    print("\t\tif(((outSeries=={}) & (~DPOutReady) & DPEarlyOutReady) | ((outSeries=={}) & DPOutReady & DPEarlyOutReady)) begin".format(HRR-1, HRR-2), file=of)
    print("\t\t\tearlyOutReady <= 1;", file=of)
    print("\t\tend", file=of)
    print("\t\telse begin", file=of)
    print("\t\t\tearlyOutReady <= 0;", file=of)
    print("\t\tend", file=of)
    print("\tend", file=of)
    print("end\n", file=of)
    
    if NMult==1:
        print("end", file=of)
        print("endgenerate\n", file=of)

In [ ]:
print("endmodule", file=of)

In [ ]:
if __Print_To_File>0:
    of.close()